In [175]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
stop = set(stopwords.words('english'))

import string
punc = set(string.punctuation)
lemma = WordNetLemmatizer()
ps = PorterStemmer()
from datetime import datetime

! pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rahul\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rahul\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Defaulting to user installation because normal site-packages is not writeable
  Using cached spacy-3.5.1-cp39-cp39-win_amd64.whl (12.2 MB)


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


Defaulting to user installation because normal site-packages is not writeable


In [25]:
# import data
webscrapped_data = pd.read_csv('C:\\Users\\rahul\\OneDrive\\Desktop\\Education\\Year 3\\Winter Sem\\ECO481\\Final Project\\webscrapped.csv')

In [9]:
webscrapped_data["date"][0].split(" ")[0] + webscrapped_data["date"][0].split(" ")[1] + webscrapped_data["date"][0].split(" ")[2]

'Nov.26,2001'

In [28]:
webscrapped_data["date"] = webscrapped_data["date"].apply(lambda x: x.strip())
webscrapped_data["date"] = webscrapped_data["date"].apply(lambda x: x.split(" ")[0] + " " + x.split(" ")[1] + " " +x.split(" ")[2])

In [85]:
top5_articlesday = webscrapped_data.groupby("date").head(5)

In [86]:
top5_articlesday.reset_index(drop = True)

,headline,date,paragraph_1,link
0,China Sets Up Unified System\nFor Central Secu...,"April 1, 2001","SHANGHAI, China -- China has set up a unified ...",https://www.wsj.com/articles/SB986114850336013671
1,Market Advances\nAs Bargain-Hunters\nLift Fall...,"April 1, 2001",Stocks rose Friday in choppy trading as invest...,https://www.wsj.com/articles/SB985954955766668512
2,Ameritrade Cuts Work Force by 7%;\nDiscloses I...,"April 1, 2001",Ameritrade Holding,https://www.wsj.com/articles/SB985989186239735853
3,Darts Beat Out Reader Picks\nAs Old Economy St...,"April 1, 2001",And here you thought,https://www.wsj.com/articles/SB985730604488524893
4,Citigroup Online-Brokerage Site\nOffers Servic...,"April 1, 2001",Many banks now offer online brokerage services...,https://www.wsj.com/articles/SB985731019140575710
...,...,...,...,...
38509,The Summer of Our Discontent,"September 4, 2011",This summer offered little respite for nervous...,https://www.wsj.com/articles/SB100014240531119...
38510,Labor Day Blues,"September 4, 2011",Two out of three unemployed Americans want the...,https://www.wsj.com/articles/SB100014240531119...
38511,South Korea’s Hanjin Shipping Files for U.S. B...,"September 4, 2016",South Korea’s,https://www.wsj.com/articles/south-koreas-hanj...
38512,How to Detect Disaster-Relief Scams,"September 4, 2017",Thomas Blaney,https://www.wsj.com/articles/how-to-detect-dis...


In [87]:

top5_articlesday["link"] = top5_articlesday["link"].apply(lambda x: x.split("/")[-1])

C:\Users\rahul\AppData\Local\Temp\ipykernel_44900\1161128878.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top5_articlesday["link"] = top5_articlesday["link"].apply(lambda x: x.split("/")[-1])


In [88]:
topics = pd.read_csv("C:\\Users\\rahul\\OneDrive\\Desktop\\Education\\Year 3\\Winter Sem\\ECO481\\ourfirstscraper\\wsj_final.csv")

In [89]:
topics.drop(["article_names", "date"], axis= 1, inplace = True)

In [90]:
topics["links"] = topics["links"].apply(lambda x: x.split("/")[-1])

In [91]:
top5_articlesday = pd.merge(top5_articlesday, topics, left_on = "link", right_on = "links", how = "left")

In [53]:
top5_articlesday.drop(["links"], axis = 1, inplace = True)

In [55]:
top5_articlesday.drop(["link"], axis = 1, inplace = True)

In [93]:
top5_articlesday.reset_index(inplace = True)


In [112]:

def clean(doc):
    if type(doc) == str:
        html_tag_free=doc.replace('<br />',' ')
        stop_free = " ".join([i for i in html_tag_free.lower().split() if i not in stop])
        numb_free = "".join([i for i in stop_free if not i.isdigit()])
        punc_free = "".join(ch for ch in numb_free if ch not in punc)
        normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
        stop_free = " ".join([i for i in normalized.lower().split() if i not in stop])
        one_letter_free = " ".join( [w for w in stop_free.split() if len(w)>1] )
    # dropping one letter words
        return one_letter_free
    else:
        return doc

top5_articlesday["cleaned_headline"] = top5_articlesday["headline"].apply(lambda x: clean(x))
top5_articlesday["cleaned_paragraph"] = top5_articlesday["paragraph_1"].apply(lambda x: clean(x))
top5_articlesday["cleaned_topic"] = top5_articlesday["topics"].apply(lambda x: clean(x))

In [115]:
top5_articlesday["date"] = pd.to_datetime(top5_articlesday["date"], infer_datetime_format=True)

In [182]:
obj = SentimentIntensityAnalyzer()

top5_articlesday["headline_sentiment"] = top5_articlesday["cleaned_headline"].apply(lambda x: obj.polarity_scores(x)['compound'])
top5_articlesday["paragraph_sentiment"] = top5_articlesday["cleaned_paragraph"].apply(lambda x: obj.polarity_scores(x)['compound'])

In [188]:
# found sentiment scores for both headlines and paragraphs
# Need the relevance of each topic based on a preset list of words


count    38514.000000
mean        -0.010931
std          0.311756
min         -0.944200
25%         -0.102700
50%          0.000000
75%          0.000000
max          0.910000
Name: headline_sentiment, dtype: float64

In [192]:
#defining finance/economy related words
finance = ["inflation", "stagflation", "interest rates", "recession", "volatility", "debt", "financial stress", "retirement", "investment", "uncertainity","auto", "saving", "credit", "bugettin", "bank account", "net worth", "loans", "growth", "exchange rate", "trade", "stock market", "walstreet", "balance of payments", "industry", "bank", "policy", "asset", "bond", "equity", "debt", "money", "currency", "economy", "finance", "financial", "market", "stock", "trade", "business", "economic", "banking", "capital", "capitalism", "capitalist", "outlook", "forecast", "prediction", "prediction", "forecast", "outlook", "forecast", 
          "pr", "public relations", "public", "relief", "real estate", "housing", "mortgage", "home", "crisis", "crash", "bubble", "bust", "recession", "depression", "recovery", "finance"]



In [208]:
def word2vec(word):
    from collections import Counter
    from math import sqrt

    # count the characters in word
    cw = Counter(word)
    # precomputes a set of the different characters
    sw = set(cw)
    # precomputes the "length" of the word vector
    lw = sqrt(sum(c*c for c in cw.values()))

    # return a tuple
    return cw, sw, lw

def cosdis(v1, v2):
    # which characters are common to the two words?
    common = v1[1].intersection(v2[1])
    # by definition of cosine distance we have
    return sum(v1[0][ch]*v2[0][ch] for ch in common)/v1[2]/v2[2]


def find_relevance(word, threshold = 0.0):
    total = []
    for j in word:
        relevance = []
        for i in finance:
            relevance.append(cosdis(word2vec(word), word2vec(i)))
        total.append(max(relevance))
    return max(total)
        


In [209]:
#try_ = top5_articlesday.groupby("date", as_index = False).agg({'cleaned_topic': list})

try_["relevance"] = try_["cleaned_topic"].apply(lambda x: find_relevance(x))

In [212]:
try_["cleaned_topic"]

0       [hong kong stock, worldwide, business finance ...
1       [tech center, canadian stock, work week, tech ...
2       [business finance asia, business finance asia,...
3       [tech brief, canadian stock, major business ne...
4       [tech center, america, washington wire, tech b...
                              ...                        
8036    [world, review outlook, review outlook, review...
8037    [review outlook, review outlook, review outloo...
8038    [review outlook, review outlook, review outloo...
8039    [review outlook, review outlook, asia, review ...
8040    [review outlook, review outlook, politics, rev...
Name: cleaned_topic, Length: 8041, dtype: object

In [214]:
! pip install torch
! pip install torchtext
import torch
import torchtext

Defaulting to user installation because normal site-packages is not writeable

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.



Defaulting to user installation because normal site-packages is not writeable


In [215]:
glove = torchtext.vocab.GloVe(name="6B", # trained on Wikipedia 2014 corpus
                              dim=50)

.vector_cache\glove.6B.zip: 862MB [08:39, 1.66MB/s]                               
100%|█████████▉| 399999/400000 [00:59<00:00, 6683.28it/s]


In [256]:
def cosine_similarity_finance(word):
    average = [] #this is the average for a given data
    for i in range(len(word)):
        if type(word[i]) == str:
            try_i = word[i] #redefine word to split
            try_i = try_i.split(" ") # split the word
            if len(try_i) > 1: #check if word can be split
                average_1 = []
                for j in try_i: #for each word in finance
                    average_2  = []
                    for k in finance:#for each word in the split word
                        try:
                            average_2.append(torch.cosine_similarity(glove[k].unsqueeze(0), glove[j].unsqueeze(0)).item()) #find the cosine similarity
                        except:
                            pass
                    average_1.append(max(average_2)) #find the max cosine similarity
            else:
                average_1 = []
                for j in finance:
                    try:
                        average_1.append(torch.cosine_similarity(glove[j].unsqueeze(0), glove[try_i[0]].unsqueeze(0)).item())
                    except:
                        pass
            average.append(max(average_1))
        else:
            average.append(0)
    return sum(average)/len(average)
    

                


       


In [257]:
try_["relevance"] = try_["cleaned_topic"].apply(lambda x: cosine_similarity_finance(x))

0       0.927433
1       0.763274
2       0.926602
3       0.814249
4       0.752776
          ...   
8036    0.874832
8037    0.749614
8038    1.000000
8039    0.947215
8040    0.941916
Name: relevance, Length: 8041, dtype: float64

In [277]:
top5_articlesday = pd.merge(top5_articlesday, try_, on = "date", how = "left")
grouped = top5_articlesday.groupby("date", as_index = False).agg({"headline_sentiment": "mean", "paragraph_sentiment": "mean", "relevance": "mean"})

In [300]:
top5_articlesday.to_csv("top5_articlesday.csv")